# Orphan Cloud and Nucleated Dwarf Aperture Photometry

## Imports


In [ ]:
# Python Imports
from pathlib import Path

# Numerical Imports
import numpy as np

# Astropy Collab Imports
from astropy import units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import QTable
from astropy.coordinates import SkyCoord
from photutils.utils import calc_total_error
from photutils.aperture import (SkyCircularAperture, SkyCircularAnnulus,
                                SkyEllipticalAperture)

# Plotting
import matplotlib.pyplot as plt

## Notebook Setup

In [ ]:
# Paths
IMG_DIR = Path('../Images/ProcessedImages/HST/DrizzledImages')

# Filters
FILTERS = ['F275W', 'F475W', 'F814W']

In [ ]:
# Aperture Coordinates
APER_CRDS = SkyCoord(
    ra=[176.1292462, 176.1300361, 176.1056656, 176.1038895, 176.1084130, 176.1058292],
    dec=[20.1455972, 20.1444086, 20.1627013, 20.1620489, 20.1672140, 20.1564409],
    unit='deg',
    frame='icrs'
)

# Apertures
APERS = [
    SkyEllipticalAperture(APER_CRDS[0], 0.944*u.arcsec, 0.717*u.arcsec, 350.346690*u.deg),
    SkyEllipticalAperture(APER_CRDS[1], 0.788*u.arcsec, 1.033*u.arcsec, 343.504620*u.deg),
    SkyEllipticalAperture(APER_CRDS[2], 1.142*u.arcsec, 1.435*u.arcsec,   0.000000*u.deg),
    SkyEllipticalAperture(APER_CRDS[3], 1.142*u.arcsec, 1.435*u.arcsec,  38.572567*u.deg),
    SkyCircularAperture(APER_CRDS[4], 0.18*u.arcsec),
    SkyCircularAperture(APER_CRDS[5], 0.18*u.arcsec)
]

# Annuli
ANNUS = [
    SkyCircularAnnulus(APER_CRDS[0], 1.25*u.arcsec, 3*u.arcsec),
    SkyCircularAnnulus(APER_CRDS[1], 1.1*u.arcsec, 1.3*u.arcsec,),
    SkyCircularAnnulus(APER_CRDS[2], 2*u.arcsec, 4*u.arcsec),
    SkyCircularAnnulus(APER_CRDS[3], 2*u.arcsec, 3*u.arcsec),
    SkyCircularAnnulus(APER_CRDS[4], 4*u.arcsec, 5*u.arcsec),
    SkyCircularAnnulus(APER_CRDS[5], 4*u.arcsec, 5*u.arcsec)
]

## Load Data

In [ ]:
# Setup Variables
imgs, hdrs, errs, wcss, zpts = {}, {}, {}, {}, {}

# Loop through Filters & Load
for filter in FILTERS:

    # Open Image
    with fits.open(IMG_DIR / f'A1367OC-{filter}_drc.fits') as hduList:

        # Get the Header, Image, and WCS
        hdrs[filter] = hduList['SCI'].header
        imgs[filter] = hduList['SCI'].data << u.electron/u.s
        errs[filter] = calc_total_error(
            imgs[filter],
            hduList['WHT'].data**(-1/2) << u.electron/u.s,
            hduList[0].header['EXPTIME'] << u.s
        )
        wcss[filter] = WCS(hdrs[filter])

        # Get the Zero Point
        zp  = -2.5 * np.log10(hdrs[filter]['PHOTFLAM']) - 21.1
        zp += 18.6921
        zp -= 5 * np.log10(hdrs[filter]['PHOTPLAM'])
        zpts[filter] = u.Magnitude(zp, unit=u.mag(u.AB/imgs[filter].unit))

## Do Photometry

In [ ]:
from astropy.visualization import simple_norm
cutout = ANNUS[3].to_pixel(wcss['F475W']).to_mask().cutout(errs['F475W'])
plt.imshow(cutout.value, origin='lower', cmap='bone', norm=simple_norm(cutout.value, 'sqrt', percent=99.5))
plt.colorbar(label='Error (e-/s)')

In [ ]:
# Create Photometry Dict
phot = {}

# Loop through Filters & Perform Aperture Photometry
for filter, img in imgs.items():

    # Get the Data Mask
    mask  = ~np.isfinite(img)
    mask |= ~np.isfinite(errs[filter])
    mask |= (errs[filter] >= 0.1*u.electron/u.s)

    # Loop through Apertures
    photTable = QTable(
        names=[
            'id', 'sky x', 'sky y', 'aper sum', 'aper err', 'aper area', 'annu sum', 'annu err',
            'annu area', 'annu mean', 'annu median'
        ],
        dtype=['i4', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8'],
        units=[
            '', 'deg', 'deg', 'electron/s', 'electron/s', 'pix2', 'electron/s', 'electron/s',
            'pix2', 'electron/s', 'electron/s'
        ]
    )
    for id, (aper, annu) in enumerate(zip(APERS, ANNUS)):

        # Start Row
        row = [id+1, aper.positions.ra, aper.positions.dec]

        # Convert to Pix
        aper = aper.to_pixel(wcs=wcss[filter])
        annu = annu.to_pixel(wcs=wcss[filter])

        # Get Aperture Photometry
        row += aper.do_photometry(img, error=errs[filter], mask=mask)
        row.append(aper.area_overlap(img)*u.pix**2)

        # Get Annulus Photometry
        row += annu.do_photometry(img, error=errs[filter], mask=mask)
        row.append(annu.area_overlap(img)*u.pix**2)

        # Get Annulus Mean & Median
        mskImg = annu.to_mask().multiply(img, np.nan)
        row.append(np.nanmean(mskImg))
        row.append(np.nanmedian(mskImg))

        # Do Background Subtraction


        # Append Row to Table
        photTable.add_row(row)

    # Do Background Subtraction
    photTable['aper sum bkg sub'] = photTable['aper sum'] - photTable['aper area']*photTable['annu sum']/photTable['annu area']
    photTable['aper err bkg sub'] = np.sqrt(
        photTable['aper err']**2 +
        (photTable['aper area']*photTable['annu err']/photTable['annu area'])**2
    )

    # Get Magnitudes
    photTable['aper mag bkg sub'] = u.Magnitude(photTable['aper sum bkg sub']) + zpts[filter]
    photTable['aper mag err bkg sub'] = u.Magnitude(
        1.0857362047581294 * photTable['aper err bkg sub'].value / photTable['aper sum bkg sub'].value
    )

    # Add to Photometry Dict
    phot[filter] = photTable


In [ ]:
# Create an Output Table of Magnitudes
outTable = QTable()
outTable['id'] = phot['F275W']['id']
outTable['sky x'] = phot['F275W']['sky x']
outTable['sky y'] = phot['F275W']['sky y']
outTable['F275W mag'] = phot['F275W']['aper mag bkg sub']
outTable['F475W mag'] = phot['F475W']['aper mag bkg sub']
outTable['F814W mag'] = phot['F814W']['aper mag bkg sub']
outTable['F275W mag err'] = phot['F275W']['aper mag err bkg sub']
outTable['F475W mag err'] = phot['F475W']['aper mag err bkg sub']
outTable['F814W mag err'] = phot['F814W']['aper mag err bkg sub']
outTable.write('A1367-OC_NDwarf-Photometry.ecsv', overwrite=True)
outTable

## Plot

### Color - Color Diagram

In [ ]:
# Get xy
x = outTable['F275W mag'] - outTable['F475W mag']
y = outTable['F475W mag'] - outTable['F814W mag']
xErr = np.sqrt(
    outTable['F275W mag err']**2 +
    outTable['F475W mag err']**2
)
yErr = np.sqrt(
    outTable['F475W mag err']**2 +
    outTable['F814W mag err']**2
)

# Get the Figure
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the Data
_ = ax.errorbar(
    x, y, xerr=xErr, yerr=yErr, fmt='.', color='C0', ecolor='k', elinewidth=1, capsize=3
)

# Annotate the Axes
for xi, yi, id in zip(x, y, outTable['id']):
    _ = ax.annotate(
        str(id), (xi.value, yi.value),
        textcoords="offset points", xytext=(4, -12),
        ha='left', va='top', fontsize=12
    )

# Label the Axes
_ = ax.set_xlabel('F275W - F475W (mag)', fontsize=14)
_ = ax.set_ylabel('F475W - F814W (mag)', fontsize=14)

# Tighten the Layout
_ = fig.tight_layout()

# Save
fig.savefig('PhotometryResults/A1367-OC_NDwarf-Photometry_F475W-F814W_vs_F275W-F475W-HighErrDrop.png', dpi=300)
fig.savefig('PhotometryResults/A1367-OC_NDwarf-Photometry_F475W-F814W_vs_F275W-F475W-HighErrDrop.pdf')

### Color - Magnitude Diagram

In [ ]:
# Get xy
x = outTable['F475W mag'] - outTable['F814W mag']
y = outTable['F475W mag']
xErr = np.sqrt(
    outTable['F475W mag err']**2 +
    outTable['F814W mag err']**2
)
yErr = outTable['F475W mag err']

# Get the Figure
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the Data
_ = ax.errorbar(
    x.value, y.value, xerr=xErr.value, yerr=yErr.value, fmt='.', color='C0', ecolor='k', elinewidth=1, capsize=3
)

# Annotate the Axes
for xi, yi, id in zip(x, y, outTable['id']):
    _ = ax.annotate(
        str(id), (xi.value, yi.value),
        textcoords="offset points", xytext=(-4, -12),
        ha='right', va='top', fontsize=12
    )

# Label the Axes
_ = ax.set_xlabel('F475W - F814W (mag)', fontsize=14)
_ = ax.set_ylabel('F475W (AB mag)', fontsize=14)

# Tighten the Layout
_ = fig.tight_layout()

# Save
fig.savefig('PhotometryResults/A1367-OC_NDwarf-Photometry_F475W_vs_F475W-F814W-HighErrDrop.png', dpi=300)
fig.savefig('PhotometryResults/A1367-OC_NDwarf-Photometry_F475W_vs_F475W-F814W-HighErrDrop.pdf')